In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

In [3]:
from argparse import Namespace

args = Namespace(
    random_state=42,
    preprocessing=True,
    test_size=0.2,
    num_queries=100,
    batch_size = 5,
    input_path="./input/feedback-prize-2021/"
)

In [5]:
from datasets import build_fdb_data, collate_fn

dataset, val, evaluator = build_fdb_data(args)

100%|██████████| 15594/15594 [00:02<00:00, 5427.35it/s]


In [8]:
from torch.utils.data import DataLoader

dl = DataLoader(dataset, args.batch_size, collate_fn=collate_fn)

docs, targets, infos = next(iter(dl))

In [9]:
num_classes = 7
predictions = {
    'pred_logits': torch.rand([args.batch_size, args.num_queries, num_classes + 1]) * torch.Tensor([[1] * num_classes + [10]]),
    'pred_boxes': torch.rand([args.batch_size, args.num_queries, 2]) * torch.Tensor([[1, 0.1]])
}

In [10]:
from models.matcher import HungarianMatcher
from models.criterion import CriterionDETR

matcher = HungarianMatcher()

weight_dict = {'loss_ce': 1., 'loss_bbox': 1., 'loss_giou': 1.}
losses = ['labels', 'boxes', 'cardinality']

criterion = CriterionDETR(num_classes, matcher=matcher, weight_dict=weight_dict,
                            eos_coef=0.1, losses=losses)

In [11]:
criterion(predictions, targets)

{'loss_ce': tensor(2.3813),
 'loss_bbox': tensor(0.0781),
 'loss_giou': tensor(0.5044),
 'cardinality_error': tensor(5.2000)}